#### Check the Directory

In [1]:
import os

In [2]:
pwd

'/work/ws-tmp/g063292-restore/g063292-projects-1736208608/Text_Summarization/Text_Summarization_Project/research'

In [3]:
cd ..

/work/ws-tmp/g063292-restore/g063292-projects-1736208608/Text_Summarization/Text_Summarization_Project


##### NOTE : code should be done In The Main /Text_Summarization_Project Directory

In [4]:
pwd

'/work/ws-tmp/g063292-restore/g063292-projects-1736208608/Text_Summarization/Text_Summarization_Project'

## Workflow

1. Update config.yaml 
2. Update params.yaml
3. Update entity 
4. Update Configuration manager in SRC config
5. Update the components
6. Update the pipeline
7. Update main.py
8. Update app.py (very last this will be done)


params.yaml will be changed

In [5]:
#03.Update Entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [6]:
#04. Update Configuration manager in SRC config

from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [8]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
Using cached accelerate-1.0.1-py3-none-any.whl (330 kB)
Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: accelerate 1.0.1
Uninstalling accelerate-1.0.1:
  Successfully uninstalled accelerate-1.0.1
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached accelerate-1.0.1-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.46.3-py3-none-any.whl (10.0 MB)
Using cached accelerate-1.0.1-py3-none-any.whl (330 kB)


In [9]:
#05. Update the components
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

/home/g063292/miniconda3/envs/TS/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-03-12 15:03:29,381: INFO: config: PyTorch version 2.4.1 available.]


In [10]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # ) 


        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        ) 

        trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [12]:
pwd

'/work/ws-tmp/g063292-restore/g063292-projects-1736208608/Text_Summarization/Text_Summarization_Project'

In [13]:
! pip install -U 'accelerate==0.26.0'

  Using cached accelerate-0.26.0-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.26.0-py3-none-any.whl (270 kB)
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.0.1
    Uninstalling accelerate-1.0.1:
      Successfully uninstalled accelerate-1.0.1


In [14]:
! pip install transformers[torch]

In [15]:
#06. Update the pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-03-12 15:03:37,782: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-12 15:03:37,786: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-12 15:03:37,788: INFO: common: created directory at: artifacts]
[2025-03-12 15:03:37,789: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyboardInterrupt: 